In [1]:
import os
import json
import glob
from tqdm import tqdm

In [2]:
train_image = "./train/img/"
train_ann = "./train/ann/"
val_image = "./valid/img/"
val_ann = "./valid/ann/"  

test_image = "./test/img/"
test_ann = "./test/ann/"

In [3]:
train_image_list = os.listdir(train_image)
train_ann_list = os.listdir(train_ann)

val_image_list = os.listdir(val_image)
val_ann_list = os.listdir(val_ann)

print(len(train_image_list), len(train_ann_list), len(val_image_list), len(val_ann_list))

1991 1991 254 254


In [4]:
test_image_list = os.listdir(test_image)
test_ann_list = os.listdir(test_ann)

print(len(test_image_list), len(test_ann_list))

250 250


In [5]:
print(train_image_list[:5])
print(train_ann_list[:5])
print(val_image_list[:5])
print(val_ann_list[:5])

['11398.jpg', '12660.jpg', '11488.jpg', '11154.jpg', '10105.jpg']
['10435.json', '11631.json', '11781.json', '13126.json', '12743.json']
['2410.jpg', '1659.jpg', '1066.jpg', '1941.jpg', '1620.jpg']
['1049.json', '251.json', '1153.json', '1009.json', '1694.json']


In [43]:
# rename image and ann
# 3449_jpg.rf.f0cbd3e5b418f05bfdab2e4bf556086e.jpg -> 3349.jpg

for i in tqdm(val_image_list):
    os.rename(val_image+i, val_image+i.split("_")[0]+".jpg")
    
# rename ann json
# 1_jpg.rf.a332191a2b0b318c103a508c515931c1.jpg.json -> 1.json
for i in tqdm(val_ann_list):
    os.rename(val_ann+i, val_ann+i.split("_")[0]+".json")

100%|██████████| 254/254 [00:00<00:00, 119313.83it/s]


In [53]:
# rename image and ann
# 1.jpg -> 10001.jpg, 1.json -> 10001.json
for i in tqdm(train_image_list):
    os.rename(train_image+i, train_image+str(int(i.split(".")[0])+10000)+".jpg")
    
for i in tqdm(train_ann_list):
    os.rename(train_ann+i, train_ann+str(int(i.split(".")[0])+10000)+".json")

100%|██████████| 1991/1991 [00:00<00:00, 125908.17it/s]


In [11]:
# for i in tqdm(test_image_list):
#     os.rename(test_image+i, test_image+str(int(i.split("_")[0])+20000)+".jpg")

for i in tqdm(test_ann_list):
    os.rename(test_ann+i, test_ann+str(int(i.split("_")[0])+20000)+".json")

100%|██████████| 250/250 [00:00<00:00, 106747.02it/s]


In [6]:
error_list = []

# check image and ann matching
for i in tqdm(test_image_list):
    if i.split(".")[0] + ".json" not in test_ann_list:
        error_list.append(i)

100%|██████████| 250/250 [00:00<00:00, 501231.36it/s]


In [7]:
print(error_list)

[]


In [12]:
# reformat ann json file to COCO dataset format
def reformat_ann(ann_path, image_path, save_path):
    with open(ann_path, 'r') as f:
        ann = json.load(f)
    with open(image_path, 'r') as f:
        image = json.load(f)
    
    new_ann = {}
    new_ann['info'] = ann['info']
    new_ann['licenses'] = ann['licenses']
    new_ann['images'] = []
    new_ann['annotations'] = []
    new_ann['categories'] = ann['categories']
    
    for i in range(len(ann['images'])):
        new_ann['images'].append(ann['images'][i])
        new_ann['annotations'].append(ann['annotations'][i])
    
    with open(save_path, 'w') as f:
        json.dump(new_ann, f)

In [13]:
import os
import json
import glob
from tqdm import tqdm

def convert_to_coco(data_dir, output_file):
    images = []
    annotations = []
    categories = [{"id": 1, "name": "tooth", "supercategory": "none"}]
    
    category_id = 1  # Since all categories are "Tooth"
    annotation_id = 1
    
    img_dir = os.path.join(data_dir, 'img')
    ann_dir = os.path.join(data_dir, 'ann')
    
    img_files = sorted(glob.glob(os.path.join(img_dir, '*.jpg')))
    
    for img_id, img_file in enumerate(tqdm(img_files), 1):
        img_name = os.path.basename(img_file)
        ann_file = os.path.join(ann_dir, os.path.splitext(img_name)[0] + '.json')

        if not os.path.exists(ann_file):
            continue
        
        with open(ann_file, 'r') as f:
            ann_data = json.load(f)
        
        # Add image information
        images.append({
            "id": img_id,
            "file_name": img_name,
            "height": ann_data['size']['height'],
            "width": ann_data['size']['width']
        })
        
        # Add annotations
        for obj in ann_data['objects']:
            points = obj['points']['exterior']
            # Convert polygon to COCO format (x, y, width, height)
            x_coords = [p[0] for p in points]
            y_coords = [p[1] for p in points]
            min_x, min_y = min(x_coords), min(y_coords)
            max_x, max_y = max(x_coords), max(y_coords)
            width, height = max_x - min_x, max_y - min_y
            
            annotations.append({
                "id": annotation_id,
                "image_id": img_id,
                "category_id": category_id,
                "segmentation": [list(sum(points, []))],  # Flatten the list of points
                "area": width * height,
                "bbox": [min_x, min_y, width, height],
                "iscrowd": 0
            })
            annotation_id += 1
    
    coco_format = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }
    
    with open(output_file, 'w') as f:
        json.dump(coco_format, f, indent=4)

In [60]:
convert_to_coco('./train/', 'train_annotations.json')

100%|██████████| 1991/1991 [00:00<00:00, 3321.81it/s]


In [61]:
convert_to_coco('./valid/', 'valid_annotations.json')

100%|██████████| 254/254 [00:00<00:00, 4019.78it/s]


In [14]:
convert_to_coco('./test/', 'test_annotations.json')

100%|██████████| 250/250 [00:00<00:00, 1808.54it/s]
